## Методы сбора и обработки данных из сети Интернет
### Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:  
    1. Наименование вакансии.  
    2. Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).  
    3. Ссылку на саму вакансию.  
    4. Сайт, откуда собрана вакансия.  
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [1]:
# загружаемые библиотеки
import json
import re
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd

In [2]:
VACANCY_NAME = 'Data scientist'
base_url = 'https://hh.ru/search/vacancy'
url = 'https://hh.ru/search/vacancy'
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 '
                  'Safari/537.36'
}
params = {
    # 'search_field': 'name',
    # 'search_field': 'company_name',
    # 'search_field': 'description',
    'text': VACANCY_NAME,
    'area': 113,
    'salary': '',
    'currency_code': 'RUR',
    'experience': 'doesNotMatter',
    'order_by': 'relevance',
    'search_period': 0,
    'items_on_page': 20,
    'no_magic': 'true',
    'L_save_area': 'true',
    'page': 0,
    # 'hhtmFrom': 'vacancy_search_list'
}

vacancy_list = []

while True:
    response = requests.get(url, headers=headers, params=params)
    if response.ok:
        dom = BeautifulSoup(response.text, 'html.parser')
        vacancy_boxes = dom.find_all('div', {'class': 'vacancy-serp-item'})

        for vacancy_box in vacancy_boxes:

            vacancy_data = {}

            vacancy = vacancy_box.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
            vacancy_name = vacancy.getText()
            vacancy_link = vacancy.get('href')
            vacancy_address = \
                vacancy_box.find('div', {'class': 'bloko-text bloko-text_no-top-indent'}).getText().split(', ')[0]

            salary = vacancy_box.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
            if salary is None:
                salary_min = None
                salary_max = None
                salary_currency = None
            else:
                salary_str = salary.getText()
                salary_list = salary_str.split(' ')
                regex = r'[\D+\\.]?$'
                if re.search(regex, salary_str):
                    salary_currency = salary_list[-1]
                else:
                    salary_currency = None
                if salary_str.startswith('от'):
                    salary_min = salary_list[1].replace('\u202f', ' ')
                    salary_max = None
                elif salary_str.startswith('до'):
                    salary_min = None
                    salary_max = salary_list[1].replace('\u202f', ' ')
                else:
                    salary_min = salary_list[0].replace('\u202f', ' ')
                    salary_max = salary_list[2].replace('\u202f', ' ')
            
            vacancy_data['Название вакансии'] = vacancy_name
            vacancy_data['Город'] = vacancy_address
            vacancy_data['Минимальная зарплата'] = salary_min
            vacancy_data['Максимальная зарплата'] = salary_max
            vacancy_data['Валюта'] = salary_currency
            vacancy_data['Ссылка'] = vacancy_link
            vacancy_data['Сайт'] = 'https://hh.ru'
            vacancy_list.append(vacancy_data)

        next_page = dom.find('a', {'data-qa': 'pager-next'})
        params['page'] += 1

    else:
        break

with open('hh.json', 'w') as file:
    json.dump(vacancy_list, file, indent=2, ensure_ascii=False)

In [3]:
df_hh = pd.DataFrame(vacancy_list)

In [4]:
display(df_hh)

,Название вакансии,Город,Минимальная зарплата,Максимальная зарплата,Валюта,Ссылка,Сайт
0,Data Scientist (команда Поиска),Москва,None,None,None,https://hh.ru/vacancy/46048820?from=vacancy_se...,https://hh.ru
1,Аналитик данных (Data scientist),Новосибирск,None,None,None,https://hh.ru/vacancy/51239177?from=vacancy_se...,https://hh.ru
2,Аналитик данных (Data scientist),Владивосток,None,None,None,https://hh.ru/vacancy/51239173?from=vacancy_se...,https://hh.ru
3,Data Scientist,Москва,600 000,1 800 000,руб.,https://hh.ru/vacancy/52631700?from=vacancy_se...,https://hh.ru
4,Data Scientist (Middle),Москва,200 000,330 000,руб.,https://hh.ru/vacancy/51421868?from=vacancy_se...,https://hh.ru
...,...,...,...,...,...,...,...
719,Business/System Administrator / L2 Engineer,Пермь,60 000,None,руб.,https://hh.ru/vacancy/51658133?from=vacancy_se...,https://hh.ru
720,Business/System Administrator / L2 Engineer,Новосибирск,60 000,None,руб.,https://hh.ru/vacancy/51658130?from=vacancy_se...,https://hh.ru
721,Business/System Administrator / L2 Engineer,Москва,60 000,None,руб.,https://hh.ru/vacancy/51658127?from=vacancy_se...,https://hh.ru
722,Business/System Administrator / L2 Engineer,Екатеринбург,60 000,None,руб.,https://hh.ru/vacancy/51658129?from=vacancy_se...,https://hh.ru
